In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance as yf

In [ ]:
    def applying_strategy_return_pct_change(df, fast_sma, slow_sma):
        df['sma_fast'] = ta.sma(close=df['Close'], length=fast_sma)
        df['sma_slow'] = ta.sma(close=df['Close'], length=slow_sma)
        df['position'] = np.where(df['sma_fast'] > df['sma_slow'], 1, np.where(df['sma_fast'] < df['sma_slow'], -1, 0))
        df['position_price'] = np.where(df['position'].shift(1) != df['position'], df['Close'], 0)
        df['position_price'] = df['position_price'].replace(0, df['position_price'], method='ffill')
        df['pct'] = np.where(df['position'].shift(1) == 0, 0, df['position_price'].pct_change())
        df['pct'] = np.where(df['position'] == 1, df['pct'] * -1, df['pct'] * 1)

# deleting 2,5 of top and bottom % of positions to reduce influence of lucky shots
        q_low = df["pct"].quantile(0.025)
        q_hi  = df["pct"].quantile(0.975)
        df_filtered = df[(df["pct"] < q_hi) & (df["pct"] > q_low)]
        pct_sum = df_filtered['pct'].sum()

# use this line for raw data without deleting outliers
#         pct_sum = df['pct'].sum()
        return pct_sum

In [ ]:
# create list with tickers you want to check

ticker_list = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'ADA-USD', 'MATIC-USD', 'OKB-USD', 
              'DOGE-USD', 'SOL-USD', 'DOT-USD']

start = '2022-01-01'
end = '2023-01-01'
interval = '1h'

In [ ]:
for ticker in ticker_list:
    
    symbol = ticker
    df = yf.download(symbol, start, end, interval=interval)
    # Preparing data for applying technicals
    df = pd.DataFrame(data=df).reset_index()
    df = df[['Open', 'High', 'Low', 'Close']]

    revenues_list = []

    for fast in range(1, 101):
        for slow in range(fast+1, 201):
            pct_change = applying_strategy_return_pct_change(df, fast, slow)
            a = {'fast': fast, 'slow': slow, 'pct_change': pct_change}
            revenues_list.append(a)

    pct_change_list = []

    for condition in revenues_list:
        pct_change_list.append(condition['pct_change'])

    max_pct_change = max(pct_change_list)

    for strategy in revenues_list:
        if strategy['pct_change'] == max_pct_change:
            print(f"{symbol} {strategy}")